Cross in Tray Function
======================

In [1]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

  Activating project at `~/globtim`


KeyError: KeyError: key Base.PkgId(Base.UUID("ee78f7c6-11fb-53f2-987a-cfe4a2b5a57a"), "Makie") not found

We define the center and range of the square domain of approximation.

In [2]:
# Constants and Parameters
n = 2
a = 10
b = 1
scale_factor = a / b
f = CrossInTray # Objective function
d = 8 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

UndefVarError: UndefVarError: `CrossInTray` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision.

In [3]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

UndefVarError: UndefVarError: `RationalPrecision` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [4]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

LoadError: LoadError: UndefVarError: `@polyvar` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at /Users/ghscholt/globtim/Examples/Notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:1

In [5]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.00125);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.00125);

UndefVarError: UndefVarError: `analyze_critical_points` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true)
display(fig_2)
# Commented out save command
# save("crossintray_lege_levelset.pdf", fig_2)

UndefVarError: UndefVarError: `cairo_plot_polyapprox_levelset` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
display(fig_2)

UndefVarError: UndefVarError: `fig_2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.